In [1]:
from __future__ import absolute_import, division, print_function
from __future__ import unicode_literals

In [2]:
import tensorflow as tf
import numpy as np
import copy
import tqdm
import IProgress
from hfunc import models
from hfunc import metrics

In [3]:
mnist = tf.keras.datasets.mnist
class_accuracy = metrics.ClassAccuracy()

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [4]:
model, r = models.train_basic_ANN(x_train, y_train, 128, (x_test, y_test))

Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2629 - accuracy: 0.9239 - val_loss: 0.1469 - val_accuracy: 0.9564
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1161 - accuracy: 0.9654 - val_loss: 0.1096 - val_accuracy: 0.9678
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0798 - accuracy: 0.9766 - val_loss: 0.0920 - val_accuracy: 0.9734
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0591 - accuracy: 0.9821 - val_loss: 0.0774 - val_accuracy: 0.9777
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0459 - accuracy: 0.9861 - val_loss: 0.0743 - val_accuracy: 0.9782
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0359 - accuracy: 0.9890 - val_loss: 0.0821 - val_accuracy: 0.9766
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0291 - accuracy: 0.9910 - val_loss: 0.0731 - val_accuracy:

In [5]:
loss, acc = model.evaluate(x_test, y_test, verbose=2)

313/313 - 1s - loss: 0.0762 - accuracy: 0.9795


In [6]:
n = 1
old = model.get_weights()

In [32]:
def remove_random_nodes(acc, loss, weights, n, to_test, x_train, y_train, v=0, remove='positive'):
    new_loss = loss
    new_acc = acc
    if remove == 'positive':
        best_score = 0
    elif remove == 'always':
        best_score = -1e20
    else:
        best_score = 0.5
    best_model = copy.deepcopy(weights)
    nodes_removed = np.array([])
    for _ in tqdm.trange(to_test):   
        new = copy.deepcopy(weights)
        to_drop = np.random.choice(len(new[1]), n, replace=False)
        for i in to_drop:
            new[0][:,i] = 0
            new[1][i] = 0
            new[2][i,:] = 0
        model.set_weights(new)
        new_loss, new_acc = model.evaluate(x_train, y_train, verbose=v)
        score = (1 - (new_loss / loss)) + ((new_acc / acc) - 1)
        if best_score < score:
            best_score = score
            best_model = copy.deepcopy(new)
            nodes_removed = to_drop.copy()
    return best_model, best_score, nodes_removed

In [8]:
to_check = 100
new_loss = loss
new_acc = acc
best_score = 0
best_model = copy.deepcopy(old)

for i in tqdm.trange(to_check):   
    new = copy.deepcopy(old)
    #for i in range(len(old)):
    #    new[i] = old[i].copy()
    to_drop = np.random.choice(len(new[1]), n, replace=False)
    # for i in to_drop:
    new[0][:,i] = 0
    new[1][i] = 0
    new[2][i,:] = 0
    model.set_weights(new)
    new_loss, new_acc = model.evaluate(x_test, y_test, verbose=0)
    score = (1 - (new_loss / loss)) + ((new_acc / acc) - 1)
    score1 = (1 - (new_loss / loss))
    score2 = ((new_acc / acc) - 1)
    if best_score < score1 or best_score < score2:
        best_score = score1
        best_model = copy.deepcopy(new)
    if best_score < score2:
        best_score = score2
        best_model = copy.deepcopy(new)

0%|          | 0/100 [00:00<?, ?it/s]


NameError: name 'check' is not defined

In [ ]:
best_model[1]

In [ ]:
model.set_weights(best_model)
loss, acc = model.evaluate(x_test, y_test, verbose=2)

In [17]:
best_score = 0
best_model = copy.deepcopy(old)
to_test = 25
for i in range(1, 65):
    temp_model, temp_score, nodes_removed = remove_random_nodes(acc, loss, old, i, to_test, x_train, y_train)
    if temp_score > best_score:
        best_model = temp_model
        best_score = temp_score
        print("Found new best model")

  4%|▍         | 1/25 [00:02<01:10,  2.93s/it]


KeyboardInterrupt: 

In [18]:
best_model[1]

array([ 0.13777253,  0.00683018, -0.04688663, -0.0542522 , -0.02874556,
       -0.06947239,  0.26529318, -0.05660995, -0.12442102, -0.00682503,
       -0.06165336,  0.04902135, -0.06425561,  0.1148253 ,  0.10713325,
       -0.03243035,  0.13046843, -0.02118045,  0.23898   ,  0.012234  ,
       -0.00759443,  0.0615023 ,  0.19653296,  0.0889185 ,  0.11125184,
        0.01821019, -0.06034907, -0.01788706, -0.22678226,  0.25768587,
       -0.00749901, -0.15829737,  0.08297709,  0.07035011,  0.19038919,
        0.21002766,  0.0498327 , -0.16142973, -0.02738184, -0.0705651 ,
       -0.05639892,  0.14290817, -0.04814705, -0.05610737,  0.02354445,
       -0.03746689,  0.07436547,  0.04749129,  0.00861628, -0.00861224,
       -0.02170092,  0.09200769,  0.15906057, -0.13180414, -0.07231998,
       -0.01664761, -0.00881143,  0.10944258,  0.00742158, -0.10521833,
        0.11596415, -0.00629094,  0.04692442,  0.00971017,  0.01094389,
       -0.02010831,  0.13656658,  0.06641621, -0.1453644 , -0.01

## Creating a restriced model

In [12]:
n = 2
best_weights, _, nodes_removed = remove_random_nodes(acc, loss, old, n, 50, x_train, y_train)

new_weights = [np.zeros((best_weights[0].shape[0], best_weights[0].shape[1] - n)), np.zeros((best_weights[1].shape[0] - n)), np.zeros((best_weights[2].shape[0] - n, best_weights[2].shape[1])), best_weights[3]]

j = 0
for i in range(len(best_weights[1])):
    if i not in nodes_removed:
        new_weights[0][:, j] = best_weights[0][:, i]
        new_weights[1][j] = best_weights[1][i]
        new_weights[2][j, :] = best_weights[2][i, :]
        j = j + 1
    
new_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128 - n, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

new_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
new_model.set_weights(new_weights)

100%|██████████| 50/50 [01:32<00:00,  1.85s/it]


In [31]:
def shrink_model(model, shrinkage_factor, x_train, y_train, size, to_test, v=0, remove='positive'):
    
    n = shrinkage_factor
    loss, acc = model.evaluate(x_train, y_train, verbose=2)
    old = model.get_weights()
    best_weights, _, nodes_removed = remove_random_nodes(acc, loss, old, n, to_test, x_train, y_train, v, remove)

    if nodes_removed.size:
        new_weights = [np.zeros((best_weights[0].shape[0], best_weights[0].shape[1] - n)), np.zeros((best_weights[1].shape[0] - n)), np.zeros((best_weights[2].shape[0] - n, best_weights[2].shape[1])), best_weights[3]]

        j = 0
        for i in range(len(best_weights[1])):
            if i not in nodes_removed:
                new_weights[0][:, j] = best_weights[0][:, i]
                new_weights[1][j] = best_weights[1][i]
                new_weights[2][j, :] = best_weights[2][i, :]
                j = j + 1

        new_model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(size - n, activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(10, activation='softmax')
        ])
    else:
        print("Shrinking unsuccessful")
        return model, size
    
    new_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    new_model.set_weights(new_weights)
    print("Shrinking successful")
    return new_model, size-n

In [14]:
new_model.evaluate(x_test, y_test, verbose=2)

313/313 - 0s - loss: 0.0752 - accuracy: 0.9791


[0.07517477869987488, 0.9790999889373779]

In [15]:
new_model.fit(x_train, y_train, epochs=3)

Epoch 1/3
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0603 - accuracy: 0.9794
Epoch 2/3
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0494 - accuracy: 0.9820
Epoch 3/3
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0471 - accuracy: 0.9834


In [16]:
new_model.evaluate(x_test, y_test, verbose=2)

313/313 - 1s - loss: 0.0702 - accuracy: 0.9811


[0.07021047919988632, 0.9811000227928162]

In [28]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
size = 128
to_test = 25
for _ in range(4):
    model.fit(x_train, y_train, epochs=1)
    model, size = shrink_model(model, 2, x_train, y_train, size, to_test)
    print(len(model.get_weights()[1]))
model.fit(x_train, y_train, epochs=1)
model.evaluate(x_test, y_test, verbose=2)

1875/1875 [==============================] - 3s 2ms/step - loss: 0.3000 - accuracy: 0.9121
1875/1875 - 2s - loss: 0.1373 - accuracy: 0.9606
100%|██████████| 25/25 [00:34<00:00,  1.38s/it]Shrinking unsuccessful
128
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1481 - accuracy: 0.9553
1875/1875 - 2s - loss: 0.0886 - accuracy: 0.9737
100%|██████████| 25/25 [00:38<00:00,  1.56s/it]Shrinking unsuccessful
128
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1129 - accuracy: 0.9656
1875/1875 - 3s - loss: 0.0639 - accuracy: 0.9809
100%|██████████| 25/25 [00:41<00:00,  1.68s/it]Shrinking unsuccessful
128
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0931 - accuracy: 0.9707
1875/1875 - 3s - loss: 0.0495 - accuracy: 0.9852
100%|██████████| 25/25 [00:41<00:00,  1.66s/it]
Shrinking successful
126
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0818 - accuracy: 0.9743
313/313 - 1s - loss: 0.0791 - accuracy: 0.9760


[0.07908758521080017, 0.9760000109672546]

In [35]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
rep = 6

best_models = []
sizes = []
scores = []
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
start_weights = copy.deepcopy(model.get_weights())
model.fit(x_train, y_train, epochs=7)
loss, acc = model.evaluate(x_test, y_test, verbose=2)
print("#############################")
best_models += [model]
scores += [(loss, acc)]
sizes +=[128]
for i in range(1, 6):
    print(f"Starting to shrinking the model by {i}")
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.set_weights(start_weights)
    size = 128
    to_test = 15
    for _ in range(rep):
        model.fit(x_train, y_train, epochs=1)
        model, size = shrink_model(model, i, x_train, y_train, size, to_test, remove='always')
    model.fit(x_train, y_train, epochs=1)
    loss, acc = model.evaluate(x_test, y_test, verbose=2)
    print("#############################")
    best_models += [model]
    scores += [(loss, acc)]
    sizes +=[(len(model.get_weights()[1]))]

Epoch 1/7
1875/1875 [==============================] - 3s 2ms/step - loss: 2.9835 - accuracy: 0.5638
Epoch 2/7
1875/1875 [==============================] - 3s 2ms/step - loss: 1.0273 - accuracy: 0.6250
Epoch 3/7
1875/1875 [==============================] - 4s 2ms/step - loss: 0.9122 - accuracy: 0.6505
Epoch 4/7
1875/1875 [==============================] - 4s 2ms/step - loss: 0.9081 - accuracy: 0.6491
Epoch 5/7
1875/1875 [==============================] - 4s 2ms/step - loss: 0.8583 - accuracy: 0.6604
Epoch 6/7
1875/1875 [==============================] - 4s 2ms/step - loss: 0.8286 - accuracy: 0.6720
Epoch 7/7
1875/1875 [==============================] - 4s 2ms/step - loss: 0.8252 - accuracy: 0.6768
313/313 - 1s - loss: 0.7109 - accuracy: 0.7263
#############################
Starting to shrinking the model by 1
1875/1875 [==============================] - 3s 2ms/step - loss: 2.8675 - accuracy: 0.5806
1875/1875 - 3s - loss: 0.9150 - accuracy: 0.6769
100%|██████████| 15/15 [00:21<00:00,  1

In [37]:
print(scores)
sizes = [128, 122, 116, 110, 104, 98]
print(sizes)

[(0.7108903527259827, 0.7263000011444092), (0.8043630123138428, 0.7213000059127808), (0.8015145659446716, 0.7307000160217285), (0.7943050861358643, 0.7063000202178955), (0.7579924464225769, 0.7103999853134155), (0.7451807856559753, 0.7310000061988831)]
[128, 122, 116, 110, 104, 98]


In [38]:
scores_plain = [scores[0]]
for i in range(1, len(scores)):
    print(f"Starting plain train of Dense size {sizes[i]}")
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(sizes[i], activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=7)
    loss, acc = model.evaluate(x_test, y_test, verbose=2)
    scores_plain += [(loss, acc)]
    print("###############################")

Starting plain train of Dense size 122
Epoch 1/7
1875/1875 [==============================] - 4s 2ms/step - loss: 2.6516 - accuracy: 0.5721
Epoch 2/7
1875/1875 [==============================] - 4s 2ms/step - loss: 0.9663 - accuracy: 0.6399
Epoch 3/7
1875/1875 [==============================] - 4s 2ms/step - loss: 0.9084 - accuracy: 0.6562
Epoch 4/7
1875/1875 [==============================] - 4s 2ms/step - loss: 0.9136 - accuracy: 0.6532
Epoch 5/7
1875/1875 [==============================] - 4s 2ms/step - loss: 0.8527 - accuracy: 0.6733
Epoch 6/7
1875/1875 [==============================] - 3s 2ms/step - loss: 0.8295 - accuracy: 0.6766
Epoch 7/7
1875/1875 [==============================] - 3s 2ms/step - loss: 0.8061 - accuracy: 0.6867
313/313 - 1s - loss: 0.6853 - accuracy: 0.7212
###############################
Starting plain train of Dense size 116
Epoch 1/7
1875/1875 [==============================] - 3s 2ms/step - loss: 2.3015 - accuracy: 0.5609
Epoch 2/7
1875/1875 [==============

In [41]:
best_models_p = []
sizes_p = []
scores_p = []
best_models_p += [model]
scores_p += [(loss, acc)]
sizes_p +=[128]
for i in range(1, 6):
    print(f"Starting to shrinking the model by {i}")
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.set_weights(start_weights)
    size = 128
    to_test = 15
    for _ in range(rep):
        model.fit(x_train, y_train, epochs=1)
        model, size = shrink_model(model, i, x_train, y_train, size, to_test, remove='positive')
    model.fit(x_train, y_train, epochs=1)
    loss, acc = model.evaluate(x_test, y_test, verbose=2)
    print("#############################")
    best_models += [model]
    scores_p += [(loss, acc)]
    sizes_p +=[(len(model.get_weights()[1]))]

Starting to shrinking the model by 1
1875/1875 [==============================] - 3s 1ms/step - loss: 3.1455 - accuracy: 0.5739
1875/1875 - 2s - loss: 0.7596 - accuracy: 0.6959
100%|██████████| 15/15 [00:16<00:00,  1.08s/it]
Shrinking successful
1875/1875 [==============================] - 2s 1ms/step - loss: 1.0185 - accuracy: 0.6220
1875/1875 - 2s - loss: 0.7235 - accuracy: 0.7375
100%|██████████| 15/15 [00:16<00:00,  1.08s/it]
Shrinking successful
1875/1875 [==============================] - 2s 1ms/step - loss: 0.9128 - accuracy: 0.6544
1875/1875 - 2s - loss: 0.6841 - accuracy: 0.7268
1875/1875 [==============================] - 2s 1ms/step - loss: 0.8814 - accuracy: 0.6667
1875/1875 - 2s - loss: 0.6386 - accuracy: 0.7567
1875/1875 [==============================] - 2s 1ms/step - loss: 0.8917 - accuracy: 0.6567
1875/1875 - 2s - loss: 0.6381 - accuracy: 0.7627
1875/1875 [==============================] - 2s 1ms/step - loss: 0.8595 - accuracy: 0.6697
1875/1875 - 2s - loss: 0.6296 - ac

In [42]:
print(sizes_p)

[128, 125, 120, 116, 116, 118]


In [45]:
print(scores)
print("#############################")
print(scores_p)
print("#############################")
print(scores_plain)

[(0.7108903527259827, 0.7263000011444092), (0.8043630123138428, 0.7213000059127808), (0.8015145659446716, 0.7307000160217285), (0.7943050861358643, 0.7063000202178955), (0.7579924464225769, 0.7103999853134155), (0.7451807856559753, 0.7310000061988831)]
#############################
[(0.7179182767868042, 0.7250000238418579), (0.6748734712600708, 0.7602999806404114), (0.9151998162269592, 0.6844000220298767), (0.7284160852432251, 0.7294999957084656), (0.7142876982688904, 0.722599983215332), (0.6719245910644531, 0.7556999921798706)]
#############################
[(0.7108903527259827, 0.7263000011444092), (0.6853137016296387, 0.7211999893188477), (0.6189708113670349, 0.7529000043869019), (0.6547967195510864, 0.7940000295639038), (0.7334647178649902, 0.7152000069618225), (0.726564884185791, 0.7024999856948853)]


In [ ]:
for i in range(len(scores)):
    print("Loss change:", (scores_plain[i][0] - scores[i][0])/scores_plain[i][0] *100, "--- Acc change:", -(scores_plain[i][1] - scores[i][1]) / scores_plain[i][1] * 100)

In [ ]:
to_remove_list = np.arange(1, 65)
num_rep = 100
loss_diff = np.zeros(num_rep)
acc_diff = np.zeros(num_rep)
loss_change = np.zeros(num_rep)
acc_change = np.zeros(num_rep)
nodes_removed_list = []
num_nodes_removed = np.zeros(num_rep)

In [ ]:
print(to_remove_list)

In [ ]:
for i in range(num_rep):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=10)
    loss, acc = model.evaluate(x_test, y_test, verbose=2)
    
    n = np.random.choice(to_remove_list, 1)
    
    best_weights, _, nodes_removed = remove_nodes(acc, loss, model.get_weights(), n, 1, x_train, y_train, 0)
    
    model.set_weights(best_weights)
    print(n)
    
    loss_new, acc_new = model.evaluate(x_test, y_test, verbose=2)
    
    loss_diff[i] = loss - loss_new
    acc_diff[i] = acc_new - acc
    loss_change[i] = loss_diff[i] / loss * 100
    acc_change[i] = acc_diff[i] / acc * 100
    num_nodes_removed[i] = n
    nodes_removed_list += [nodes_removed]

In [ ]:
for i in range(1, 65):
    print(f"{i} nodes removed")
    print("Loss changes:",loss_change[num_nodes_removed == i])
    print("Accuracy changes:",acc_change[num_nodes_removed == i])
    print("#########################")